# Assess performance of `search_ans` function

In [30]:
from helper import init_ipynb
envfound = init_ipynb()

In [31]:
import pickle
from evaluation import MCQBenchmark
import os
from misc import load_pickle
from IPython.display import clear_output
import time

DIR = os.environ["DIR_PATH"] if envfound else None
DEVICE = os.environ["DEVICE"] if envfound else None

### Test data


The response here were extract with the classical `search_ans` function defined before.

In [32]:
def search_ans(pred):
    if(type(pred) != str):
        pred = pred.outputs[0].text
    ans_sentence = [x for x in pred.split(".") if "Therefore" in x]
    

    if(ans_sentence == []):
        return "-1"
    sel = [l for l in ["A", "B", "C", "D", "E"] if l in ans_sentence[0]]
    return sel[0] if sel != [] else "-1"


In [33]:
gens = load_pickle("docs/generations_llm/aes7/epfl-llm-meditron-7b.aes7.pkl")

Associated benchmark :  aes7


In [5]:
pct = 0
yn = ""
for gen in gens:
    clear_output(wait=True)
    print("*"*150)
    print(gen[0])
    print("Ground truth answer :\n", gen[1])
    print("Generated answer :\n", gen[2])
    print("Classic extract :\n", search_ans(gen[2]))
    time.sleep(1)
    while(yn not in ["y", "n"]):
        yn = input("Does generated answer and classic extract match ? y/n")
    if(yn == "q"):
        break
    pct += 1 if yn == "y" else 0
    yn = ""

******************************************************************************************************************************************************
Based on the ILAE consensus classification, which of the following findings are seen in the most common histopathological subtype of hippocampal sclerosis?
A. Neuronal cell loss in the CA2 and CA3 regions
B. Neuronal cell loss in the CA1 and CA3 regions
C. Neuronal cell loss in the CA1 and CA4 regions
D. Neuronal cell loss in the dentate regions
Ground truth answer :
 C
Generated answer :
  The most common histopathological subtype of hippocampal sclerosis is characterized by neuronal cell loss in the CA1 and CA3 regions. Other less common subtypes include neuronal cell loss in the dentate regions, which is seen in 20% of patients with hippocampal sclerosis. Neuronal cell loss in the CA2 and CA3 regions is seen in approximately 15% of patients with hippocampal sclerosis, while neuronal cell loss in the CA1 and CA4 regions is seen only ra

In [8]:
print(pct, "% of accuracy of the extraction with classic extract")

89 % of accuracy of the extraction with classic extract


### Classic search_ans v2

In [11]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     /home/antoinemagron/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [48]:
FLAGS = ["Therefore", "correct", "answer"]
def match_score(sentence):
    return len([flag for flag in FLAGS if flag in sentence]) / len(FLAGS)

def simple_extract(ans_sentence):
    sel = [l for l in ["A", "B", "C", "D", "E"] if l in ans_sentence]
    return sel[0] if sel != [] else "-1"

def csa2(pred):
    sent_text = nltk.sent_tokenize(pred.replace("\n", ";"))
    sentence, score = sorted([[sent, match_score(sent)] for sent in sent_text], key=lambda _: _[1], reverse=True)[0]
    if(score == 0):
        return "-1"
    return simple_extract(sentence)

In [49]:
csa2(gens[0][2])

'D'

In [50]:
for i, gen in enumerate(gens):
    print("*"*50 + " "*5 + str(i) + " "*5 + "*"*50)
    print("Generated answer :\n", gen[2])
    print("Classic extract :\n", csa2(gen[2]))

**************************************************     0     **************************************************
Generated answer :
  The most common histopathological subtype of hippocampal sclerosis is characterized by neuronal cell loss in the CA1 and CA3 regions. Other less common subtypes include neuronal cell loss in the dentate regions, which is seen in 20% of patients with hippocampal sclerosis. Neuronal cell loss in the CA2 and CA3 regions is seen in approximately 15% of patients with hippocampal sclerosis, while neuronal cell loss in the CA1 and CA4 regions is seen only rarely.
Therefore, the correct answer is  D.
Classic extract :
 D
**************************************************     1     **************************************************
Generated answer :
  The RNS System is a brain-responsive neurostimulation device that delivers electrical stimulation to the brain in response to seizure activity. It is approved for use in patients with medically refractory focal epil

In [42]:
s = """    The minimum seizure-free period required to obtain or maintain a driver's license for people with epilepsy varies by state in the US. What is the minimum requirement to obtain a commercial driver's license (CDL) for a person with epilepsy, as specified by the US Department of Transportation?
A. Seizure free on medication for ten years
B. Seizure free on or off medication for six months
C. Seizure free off medication for ten years
D. Seizure free off medication for two years
Therefore, the correct answer is  C."""

sent_text = nltk.sent_tokenize(s.replace("\n", "."))
sentence, score = sorted([[sent, match_score(sent)] for sent in sent_text], key=lambda _: _[1], reverse=True)[0]
print(sentence)
print(csa2(sentence))

Seizure free off medication for two years.Therefore, the correct answer is  C.
C
